In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
DIR = '/home/tfir/Log/Player_information/'
flist = os.listdir(DIR)
flist.sort()
del flist[-2:]
flist

['log_20100508.csv',
 'log_20100509.csv',
 'log_20100510.csv',
 'log_20100511.csv',
 'log_20100512.csv',
 'log_20100513.csv',
 'log_20100514.csv',
 'log_20100515.csv',
 'log_20100516.csv',
 'log_20100517.csv']

In [3]:
for s in flist:
    fname = DIR + s
    if s == 'log_20100508.csv':
        df1 = pd.read_csv(fname)
        max1 = df1['max_level']
        max1.index = df1['actor_account']
    else:
        df2 = pd.read_csv(fname)
        max2 = df2['max_level']
        max2.index = df2['actor_account']
        max1 = pd.merge(max1, max2, on='actor_account', how='outer').fillna(0).max(axis=1)
        max1.name = 'max_level'
max1 = max1.sort_index()
max1

actor_account
1000115    43.0
1001198    10.0
1001279    50.0
1002212    12.0
1002538    23.0
           ... 
1998898    21.0
1999092    36.0
1999487    50.0
1999778    50.0
1999807    32.0
Name: max_level, Length: 3609, dtype: float64

In [4]:
drop_columns = ["login_count_x", "login_count_y", 
                "logout_count_x", "logout_count_y", 
                "login_day_count_x", "login_day_count_y",
                "money_up_x", "money_up_y",
                "money_down_x", "money_down_y",
                "play_time_x", "play_time_y",
                "ip_count_x", "ip_count_y"]
for s in flist:
    fname = DIR + s
    if s == 'log_20100508.csv':
        df = pd.read_csv(fname)
        del df['max_level']
    else:
        df2 = pd.read_csv(fname)
        del df2['max_level']
        df = pd.merge(df, df2, on='actor_account', how='outer').fillna(0)
        df["login_count"] = df["login_count_x"] + df["login_count_y"]
        df["logout_count"] = df["logout_count_x"] + df["logout_count_y"]
        df["login_day_count"] = df["login_day_count_x"] + df["login_day_count_y"]
        df["money_up"] = df["money_up_x"] + df["money_up_y"]
        df["money_down"] = df["money_down_x"] + df["money_down_y"]
        df["play_time"] = df["play_time_x"] + df["play_time_y"]
        df["ip_count"] = df["ip_count_x"] + df["ip_count_y"]
        df.drop(drop_columns, axis=1, inplace=True)

df = df.sort_values(by='actor_account').reset_index(drop=True)
df

,actor_account,login_count,logout_count,login_day_count,money_up,money_down,play_time,ip_count
0,1000115,10.0,10.0,8.0,3871916.0,2900110.0,92718.0,10.0
1,1001198,21.0,21.0,8.0,83516609.0,85025880.0,12156.0,21.0
2,1001279,8.0,9.0,4.0,5404049.0,5543516.0,90260.0,8.0
3,1002212,10.0,10.0,6.0,1550855.0,365913.0,46250.0,10.0
4,1002538,8.0,8.0,3.0,424729.0,297222.0,56572.0,8.0
...,...,...,...,...,...,...,...,...
3604,1998898,31.0,31.0,10.0,324547082.0,439634094.0,93594.0,31.0
3605,1999092,46.0,46.0,5.0,7917170.0,6733972.0,66637.0,46.0
3606,1999487,145.0,145.0,10.0,51404094.0,94099748.0,443838.0,145.0
3607,1999778,83.0,82.0,9.0,46707946.0,120577495.0,241821.0,83.0


In [5]:
df["money_change"] = df["money_up"] - df["money_down"]
df["avg_money"] = df["money_change"] / df["play_time"]
df.drop(["money_up", "money_down", "money_change"], axis=1, inplace=True)


In [6]:
df = df[["actor_account", "login_count", "logout_count", "login_day_count", "play_time", "avg_money", "ip_count"]]
df

,actor_account,login_count,logout_count,login_day_count,play_time,avg_money,ip_count
0,1000115,10.0,10.0,8.0,92718.0,10.481309,10.0
1,1001198,21.0,21.0,8.0,12156.0,-124.158523,21.0
2,1001279,8.0,9.0,4.0,90260.0,-1.545170,8.0
3,1002212,10.0,10.0,6.0,46250.0,25.620368,10.0
4,1002538,8.0,8.0,3.0,56572.0,2.253889,8.0
...,...,...,...,...,...,...,...
3604,1998898,31.0,31.0,10.0,93594.0,-1229.640917,31.0
3605,1999092,46.0,46.0,5.0,66637.0,17.755871,46.0
3606,1999487,145.0,145.0,10.0,443838.0,-96.196482,145.0
3607,1999778,83.0,82.0,9.0,241821.0,-305.472019,83.0


In [7]:
Player_information = pd.merge(df, max1, on='actor_account', how='outer').fillna(0)
Player_information

,actor_account,login_count,logout_count,login_day_count,play_time,avg_money,ip_count,max_level
0,1000115,10.0,10.0,8.0,92718.0,10.481309,10.0,43.0
1,1001198,21.0,21.0,8.0,12156.0,-124.158523,21.0,10.0
2,1001279,8.0,9.0,4.0,90260.0,-1.545170,8.0,50.0
3,1002212,10.0,10.0,6.0,46250.0,25.620368,10.0,12.0
4,1002538,8.0,8.0,3.0,56572.0,2.253889,8.0,23.0
...,...,...,...,...,...,...,...,...
3604,1998898,31.0,31.0,10.0,93594.0,-1229.640917,31.0,21.0
3605,1999092,46.0,46.0,5.0,66637.0,17.755871,46.0,36.0
3606,1999487,145.0,145.0,10.0,443838.0,-96.196482,145.0,50.0
3607,1999778,83.0,82.0,9.0,241821.0,-305.472019,83.0,50.0


In [63]:
label = pd.read_csv("/home/tfir/dataset/labeled_account.csv")
label.columns = ["actor_account", "class"] 
label

,actor_account,class
0,1000127,0
1,1000344,0
2,1000779,1
3,1000949,1
4,1001067,0
...,...,...
5395,1999339,0
5396,1999462,0
5397,1999554,0
5398,1999606,1


In [64]:
Player_information = pd.merge(Player_information, label, on='actor_account', how='outer').fillna(0)

In [65]:
Player_information

,actor_account,login_count,logout_count,login_day_count,play_time,avg_money,ip_count,max_level,class
0,1000127,10.0,10.0,4.0,62134.0,-220.922313,10.0,50.0,0
1,1000344,17.0,17.0,3.0,91998.0,51.823833,17.0,50.0,0
2,1000779,11.0,12.0,4.0,302896.0,-1.052312,11.0,35.0,1
3,1000949,8.0,9.0,4.0,347198.0,-0.014966,8.0,30.0,1
4,1001067,20.0,20.0,8.0,555136.0,21.621149,20.0,43.0,0
...,...,...,...,...,...,...,...,...,...
5395,1999339,19.0,19.0,3.0,24482.0,-16.576669,19.0,42.0,0
5396,1999462,19.0,19.0,6.0,84656.0,-27.136883,19.0,37.0,0
5397,1999554,5.0,5.0,2.0,21023.0,232.327546,5.0,17.0,0
5398,1999606,33.0,34.0,3.0,262127.0,-40.240467,33.0,50.0,1


In [8]:
Player_information.to_csv("/home/tfir/KJS/FinalLog/testLog/Player_information.csv", index=False, encoding='utf8')